# Code for Lecture 3 Hands on Session

In [1]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

## Transportation problem

A company has three PC assembly plants at locations 1, 2, and 3, with monthly production capacity of 1700 units, 2000 units, and 1700 units, respectively. Their PC's are sold through four retail outlets in locations A, B, C, and D, with monthly orders of 1700 units, 1000 units, 1500 units, and 1200 units respectively. The shipping costs from each plant to each outlet are presented in the following table. Use a linear programming model to find out the optimal shipping decision.  

| Shipping cost | A | B | C | D  |
|---------------|---|---|---|----|
| 1             | 5 | 3 | 2 | 6  |
| 2             | 7 | 7 | 8 | 10 |
| 3             | 6 | 5 | 3 | 8  |

In general, this model can be formulated as:

$$
\begin{array}{rll}
\min\ & \displaystyle \sum_{i \in P} \sum_{j \in R} c_{ij}x_{ij} &\\
\mbox{s.t.}\ & \displaystyle \sum_{i \in P} x_{ij} \geq d_j & \forall j \in R\\
&\displaystyle \sum_{j \in R} x_{ij} \leq s_i & \forall i \in P \\
&x_{ij} \geq 0 &\forall i \in P, j \in R
\end{array}
$$
where $P$ and $R$ denotes the sets of all plants and retail outlets, respectively, $c_{ij}$ the transporation cost between plant $i$ and retail outlet $j$, $d_j$ the demand in retail outlet $j$, $s_i$ production capacity of plant $i$, for all $i \in P, j \in R$. 

The data is in the file "Transportation.csv"


In [2]:
data1 = pd.read_csv("Transportation.csv") # the datafile and the code should be in the same folder
print( data1 )

data = data1.values

plants, capacity, outlet, demand, cost = \
data[0:3,0], data[0:3,1].astype(np.float64), data[:,2],data[:,3].astype(np.float64), data[0:3,6:10].astype(np.float64)  
plants, capacity, outlet, demand, cost

   Plants  Capacity Outlets  Demand  Unnamed: 4  Shipping cost    A    B    C  \
0     1.0    1700.0       A    1700         NaN            1.0  5.0  3.0  2.0   
1     2.0    2000.0       B    1000         NaN            2.0  7.0  7.0  8.0   
2     3.0    1700.0       C    1500         NaN            3.0  6.0  5.0  3.0   
3     NaN       NaN       D    1200         NaN            NaN  NaN  NaN  NaN   

      D  
0   6.0  
1  10.0  
2   8.0  
3   NaN  


(array([1.0, 2.0, 3.0], dtype=object),
 array([1700., 2000., 1700.]),
 array(['A', 'B', 'C', 'D'], dtype=object),
 array([1700., 1000., 1500., 1200.]),
 array([[ 5.,  3.,  2.,  6.],
        [ 7.,  7.,  8., 10.],
        [ 6.,  5.,  3.,  8.]]))

In [3]:
p,r = cost.shape
## claim a model
m1 = ro.Model("whatever")

## define dv
x = m1.dvar( (p,r), "I" )

## Obj
m1.min( sum( cost[i,j]* x[i,j] for i in range(p) for j in range(r) ) )

## constraints
m1.st( sum( x[i,j] for i in range(p) ) >= demand[j] for j in range(r) )
m1.st( sum( x[i,j] for j in range(r) ) <= capacity[i] for i in range(p) )
m1.st( x[i,j] >= 0 for i in range(p) for j in range(r) )

## solve the model
m1.solve(grb)

## obtain the solutions
print( "Shipping amount is: \n", x.get() )
print( "The total cost is: ", m1.get() )


Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-14
Being solved by Gurobi...
Solution status: 2
Running time: 0.0085s
Shipping amount is: 
 [[  -0.  700.    0. 1000.]
 [1700.  300.   -0.   -0.]
 [   0.   -0. 1500.  200.]]
The total cost is:  28200.0


### Matrix form

In [4]:
p,r = cost.shape
## claim a model
S01 = ro.Model("whatever")

## define dv
x = S01.dvar( (p,r), "I" )

## Obj
S01.min( (cost * x).sum() )

## constraints
S01.st( x.sum(axis = 0) >= demand, 
       x.sum(axis = 1) <= capacity,
       x >= 0 )

## solve the model
S01.solve(grb)

## obtain the solutions
print( "Shipping amount is: \n", x.get() )
print( "The total cost is: ", S01.get() )


Being solved by Gurobi...
Solution status: 2
Running time: 0.0038s
Shipping amount is: 
 [[  -0.  700.    0. 1000.]
 [1700.  300.   -0.   -0.]
 [   0.   -0. 1500.  200.]]
The total cost is:  28200.0


# LP： Investment Stock

* You purchased $s_i$ shares of stock $i$ at price $p_i, i = 1,…,n$
* Current price of stock $i$ is $q_i$
* You expect that the price of stock $i$ one year later will be $r_i$
* You pay a capital gain tax at the rate of 30\% on any capital gains at the time of sale
* You want to raise $K$ amount of cash after taxes
* You pay 1% in transaction costs
* Example: You sell 1,000 shares at SGD 50 per share; you have bought them at SGD 30; Net Cash is,
$$
50 \times 1000 - 0.3 \times (50-30)\times 1000 - 0.01\times 50 \times 1000 = 43,500 
$$


* Decision variables: $x_i$ the amount of share $i$ to sell

* Objective: Maximize the expected return (next year)
$$
\sum_{i=1}^{N}r_i(s_i-x_i)
$$

* Constraints: Able to raise the fund $K$.
$$
\sum_{i=1}^{N}(q_i x_i - 0.3\max(q_i-p_i,0)x_i-0.01x_iq_i) \ge K
$$


The model is LP:
\begin{equation}
\begin{array}{ll}
\text{maximize} &  \sum_{i=1}^{N}r_i(s_i-x_i)\\
\text{subject to} &  \sum_{i=1}^{N}(q_i x_i - 0.3\max(q_i-p_i,0)x_i-0.01x_iq_i) \ge K\\
 & 0\le x_i\le s_i, \forall i = 1,...,N
\end{array}
\end{equation}
Default: K=9000

The data is in the file "data1.csv"

In [5]:
data1 = pd.read_csv('data1.csv')
print( data1 )

  Stock  Buying Price  Current Share  Current Price  Expected Future Price
0    S1           1.2           1000            2.1                    2.0
1    S2           2.1           1000            3.2                    3.7
2    S3           3.2           1000            4.1                    5.2
3    S4           4.1           1000            5.1                    7.1
4    S5           4.5           1000            6.7                    9.1


In [8]:
# build model 
data1 = pd.read_csv('data1.csv')
header = data1.columns
data = data1.values
stock_name, buy_price,cur_share, cur_price, exp_price  = data[:,0], data[:,1],data[:,2],data[:,3], data[:,4] 

# Amount to raise
K = 9000

# Transaction cost (in fraction)
tCost = 0.01

# Capital gain Tax (in fraction)
capitalGainTax = 0.3

# Number of different stocks
n = len(data)

In [10]:
# Preparing an optimization model
model = ro.Model('Stocks')

# x[i]: Amount of stock i to sell
x = model.dvar( n ) 

# Objective function
model.max( sum((cur_share[i]-x[i])*exp_price[i] for i in range(n)) )
#model.max((cur_share-x)@exp_price)


# Setting Constraints
model.st( x[i] <= cur_share[i] for i in range(n) ) #Ensures that short-selling does not occur
model.st( sum(x[i]*cur_price[i]-x[i]*cur_price[i]*tCost-capitalGainTax*x[i]*max(cur_price[i]-buy_price[i],0) for i in range(n)) >= K )
model.st( x >= 0 ) #Non-negative

# Solve model
model.solve( grb )

print( x.get() )

print( model.get() )

Being solved by Gurobi...
Solution status: 2
Running time: 0.0014s
[1000.        1000.        1000.         118.7618446    0.       ]
15356.790903348072


In [11]:
# build model 
data1 = pd.read_csv('data1.csv')
data = data1.values
stock = data[:,0]
# pay attention to the differences
stock_name, buy_price,cur_share, cur_price, exp_price  = \
data[:,0], data[:,1].astype(np.float64),data[:,2].astype(np.float64),data[:,3].astype(np.float64), data[:,4].astype(np.float64) 

In [12]:
K = 9000 # set target

m1 = ro.Model("investlp")
sell = m1.dvar(len(stock))

m1.max( exp_price @ (cur_share - sell) )

m1.st( (sell @ cur_price - 0.3* sell @ np.maximum(cur_price - buy_price,0) - 0.01*sell @ cur_price).sum() >= K )
m1.st( sell >= 0, sell <= cur_share )

m1.solve( grb )

print( sell.get() )

print( m1.get() )

Being solved by Gurobi...
Solution status: 2
Running time: 0.0010s
[1000.        1000.        1000.         118.7618446    0.       ]
15356.790903348072


## LP： InvestmentBonds
* Invest amount \$K on N bonds over T periods.
* Cash earns a fixed return per year
* Each bond pays an interest rate that compounds each year, and pays the principal plus compounded interest at the end of a maturity period. 
* Each bond has a maximum invest limit.
* Goal is to maximize the final wealth. 

Examples: $K = \$ 1$ M, N=4, T=5, Cash interest rate = 2%

| Bond | Available Year | Maturity Period | Annual Interest Rate  | Limit   |
|------|----------------|-----------------|-----------------------|---------|
| B1   | 1              | 4               | 0.03                  | 1,000,000 |
| B2   | 5              | 1               | 0.04                  | 200,000  |
| B3   | 2              | 4               | 0.05                  | 500,000  |
| B4   | 2              | 3               | 0.06                  | 200,000  |


Let $y_t$ be cash investment at the beginning of the $t$th year and $x_i$ be the investment in the $i$th bound, we then have 

\begin{align*}
\max \ & 1.05 y_5 + 1.04 x_2 + 1.06^4 x_3 \\
\mbox{s.t.}\ & y_1 + x_1 = 1,000,000 \\
&y_2 + x_3 + x_4 = 1.02 y_1 \\
&y_3 = 1.02 y_2 \\
&y_4 = 1.02 y_3 \\
&y_5 + x_2 = 1.02 y_4 + 1.03^4x_1 + 1.06^3 x_4\\
&0 \leq x_1 \leq 1,000,000\\
&0 \leq x_2 \leq 200,000\\
0 \leq x_3 \leq 500,000\\
0 \leq x_4 \leq 500,000\\
\end{align*}


The data is in the file "data2.csv"

In [11]:
# Investment in bonds 

# Declaring Parameters
capital = 1000000    # Investment capital
cashRate = 0.02     # Cash Intereset rate
cashRet = (1.0 + cashRate)  # Cash return

# Read from data file 
data2 = pd.read_csv("data2.csv")
header = data2.columns
data = data2.values
print(data2)

#  Extract data to variables and Calculate bond return
bondName, bondStYear, bondDuration, bondRate, bondLimit = \
data[:,0], data[:,1],data[:,2],data[:,3], data[:,4]

# Number of different bonds available
n = len(bondName)

# Number of years 
years = max(bondStYear + bondDuration)

print(years)

  Bond  Available Year  Maturity Period  Annual Interest Rate     Limit
0   B1               1                4                   0.03  1000000
1   B2               5                1                   0.04   200000
2   B3               2                4                   0.05   500000
3   B4               2                3                   0.06   200000
6


In [12]:
# Preparing an optimization model
model = ro.Model('Investment')

# x[i]: Amount of bond i to buy
x = model.dvar(n)

# x[t]: Amount go to bank in year t
y = model.dvar(years)

# Setting objective
model.max( 1*y[years-1] )

# Setting constraints
model.st( y[0] + sum( x[i] for i in range(n) if bondStYear[i] == 1 ) == capital )
for t in range(1,years):
    model.st( y[t] + sum(x[i] for i in range(n) if bondStYear[i] == t+1) == \
             cashRet*y[t-1] + sum(x[i]*((bondRate[i] + 1)**bondDuration[i]) \
                                for i in range(n) if (bondStYear[i] + bondDuration[i] == t+1)))
model.st( x[i] <= bondLimit[i] for i in range(n) )
model.st( x >= 0, y >= 0)

# Solve model
model.solve(grb)

print(x.get())
print(y.get())
print(model.get())

Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
[313725.49019608 200000.         500000.         200000.        ]
[ 686274.50980392       0.               0.               0.
  391304.00313725 1214883.2082    ]
1214883.2082000002
